IGN review classification by BERT 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/minman28/QTM-340/main/IGNposneg2.csv', delimiter=',',index_col=0)

In [3]:
df

,review,score
1,Makes the original look like the prototype it ...,1
2,Featuring the same kind of thrilling freedom o...,1
3,If the graphical boost isn't enough - and let'...,1
4,"The voice acting, the challenges, the amazing ...",1
5,With an expertly balanced difficulty progressi...,1
...,...,...
1444,There’s no doubt that City of Brass is meant t...,0
1445,Secret of Mana HD Remaster is a bit of an odd ...,1
1446,"Clumsy, superficial, and in many ways fundamen...",0
1447,While its bold and detailed environments depic...,0


In [4]:
df.groupby('score').count()

,review
score,
0,104
1,1344


In [5]:
#I sampled 100 negative and positive reviews to create the final dataframe for this test
df_pos = df[df['score'] == 1]
df_neg = df[df['score'] == 0]
df_neg = df_neg.sample(n=100)
df_pos = df_pos.sample(n=100)

In [6]:
df_final = pd.concat([df_pos, df_neg])
df_final

,review,score
1306,Black Ops 4’s rough edges do hold it back from...,1
383,PES 2014 looks better than ever and the gamepl...,1
91,What brings the game down considerably is the ...,1
465,Though Titanfall is somewhat regrettably a bar...,1
56,Mighty Switch Force is a great first effort fr...,1
...,...,...
108,"The current game is broken, buggy, and not ver...",0
267,A prime example of a game that oversells and u...,0
855,Codemasters' F1 2015 racer falls far behind th...,0
647,Enemy Front's attempt to show a different side...,0


In [7]:
df_final.groupby('score').count()

,review
score,
0,100
1,100


In [8]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [9]:
tokenized = df_final['review'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [10]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [11]:
np.array(padded).shape

(200, 119)

In [12]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(200, 119)

In [13]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [14]:
features = last_hidden_states[0][:,0,:].numpy()

In [15]:
labels = df_final['score']

In [16]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [17]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)

/data/jupyterhub/venv/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression()

In [18]:
lr_clf.score(test_features, test_labels)

0.84

In [19]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.433 (+/- 0.11)


/data/jupyterhub/venv/lib/python3.6/site-packages/sklearn/dummy.py:132: FutureWarning: The default value of strategy will change from stratified to prior in 0.24.
  "stratified to prior in 0.24.", FutureWarning)
